# 2D Scenegraph Example

From first principles

Let's start at the beginning: we need to draw lines on the screen. While Pillow offers functions for drawing lines, we'll implement a simple one ourselves. (The first 26 cells jut repeat stuff you should already know)

In [ ]:
from PIL import Image

In [ ]:
def naive_line(img, p1, p2, color=255):
    vertical = False 
    w, h = img.size # new: remember the size of the image
    x1, y1 = p1
    x2, y2 = p2
    if abs(y2-y1) > abs(x2-x1):
        vertical = True
        x1, y1 = y1, x1
        x2, y2 = y2, x2
    if x1 > x2:
        x1, x2 = x2, x1
        y1, y2 = y2, y1
    if x2 == x1: # handle line of length 0
        return img
    m = (y2-y1) / (x2-x1)
    t = y1 - m*x1
    for x in range(x1, x2+1):
        y = int(m*x + t)
        if vertical:
            x, y = y, x
        if (x <= w+1 and y <= h-1): # new: only draw pixels within the canvas
            img.putpixel((x,y), color) 
    return img

In [ ]:
Image.Image.line = naive_line
i = Image.new("L",(100,100))
i.line((10,10), (30,70)) # -> naive_line(i, (10,10), (30,70))

This only works because 

- you can dynamically add methods to classes (``Image.Image.line = naive_line``)
- methods are not really different than normal functions - 
  Python just translates calls to ``obj.line(p1, p2)`` to ``line(obj, p1, p2)``
- the first parameter of `ǹaive_line()`` (``img``) expects an ``Image`` object - and the aforementioned mechanism
  results in a call of `ǹaive_line(i, p1, p2)`` - i.e., passes our image object to the function
  
We won't use this method in the following, though.

## Drawing Shapes

We'll just draw unfilled shapes for now because filled shapes are actually a little bit more difficult

In [ ]:
# a generator that translates a tuple (p1, p2, p3) into a series of tuples (p1, p2), (p2, p3), (p3,p1)
def pairwise_wrap(l):
    l = iter(l)
    first = next(l, None)
    prev = first
    while o := next(l, False):
        yield (prev, o)
        prev = o
    yield (prev, first)

In [ ]:
def draw_shape(img, shape, color=255):
    for p1, p2 in pairwise_wrap(shape):
        naive_line(img, map(int, p1), map(int, p2), color)
    return img
        
Image.Image.draw_shape = draw_shape # also assign this function to the Image class

In [ ]:
i = Image.new("L",(100,100))
rect = ((10,10), (10,90), (90,90), (90,10))
i.draw_shape(rect)

## Transformations

In [ ]:
import numpy as np
from numpy import matrix as M
from math import sin, cos, pi

In [ ]:
A = M('1 0; 0 1')
B = M([[-1, 0], [1,0]])
A, B

In [ ]:
v = (2,3)

In [ ]:
A * v # this does not work 

In [ ]:
A * 3 # this does work

In [ ]:
B @ v

In [ ]:
v @ B

In [ ]:
def translate(tx, ty, p=None):
    T = M([[1, 0, tx],
           [0, 1, ty],
           [0, 0, 1]])
    if p is None:
        return T
    else:
        p = list(p)
        p.append(1)
        p = T @ p
        x = p.tolist()[0][0]
        y = p.tolist()[0][1]
        return((x,y))

In [ ]:
translate(0, 0)

In [ ]:
def rotate(angle, p=None):
    # degree to radian
    angle = angle / (180/pi)
    R = M([[cos(angle), -sin(angle), 0],
          [sin(angle), cos(angle), 0],
          [0, 0, 1]])
    if p is None:
        return R
    else:
        p = list(p)
        p.append(1)
        p = R @ p
        x = p.tolist()[0][0]
        y = p.tolist()[0][1]
        return((x,y))

In [ ]:
rotate(45, (2,2))

In [ ]:
def scale(sx, sy, p=None):
    S = M([[sx, 0, 0],
           [0, sy, 0],
           [0, 0, 1]])
    if p is None:
        return S
    else:
        p = list(p)
        p.append(1)
        p = S @ p
        x = p.tolist()[0][0]
        y = p.tolist()[0][1]
        return((x,y))

In [ ]:
scale(3, -0.3, (20, 5))

In [ ]:
rotate(180, [10,0])

In [ ]:
i = Image.new("L",(100,100))
for a in range(0, 360, 5):
    _ = i.line(map(int, translate(50,50, rotate(a, [30,0]))), 
               map(int, translate(50,50, rotate(a, [40,0]))))
display(i)

In [ ]:
A = translate(20,20)
B = rotate(45)
print(A @ B)

In [ ]:
C = A @ B
C @ (10,10, 1)

### Helper Functions

Let's define two helper functions that make transforming and drawing shapes easier.

In [ ]:
def draw_shape(img, shape, color=255):
    for p1, p2 in pairwise_wrap(shape):
        naive_line(img, map(int, p1), map(int, p2), color)

In [ ]:
def transform(shape, matrix):
    ret = []
    for point in shape:
        if len(point) == 2:
            point = list(point)
            point.append(1)
        elif len(point) != 3:
            raise ValueError("Points need to have a length of 2 or 3.")
        p = matrix @ point
        x = p.tolist()[0][0]
        y = p.tolist()[0][1]
        ret.append((x,y))
    return ret

In [ ]:
i = Image.new("L",(100,100))
rect = ((10,10), (10,90), (90,90), (90,10))
rect2 = transform(rect, translate(50,50) @ rotate(45) @ scale(0.5, 0.5) @ translate(-50,-50))
i.draw_shape(rect2)

## Finally, a scenegraph

First step: a very shallow scenegraph where the root node has only transform nodes as children - which in turn have exactly one shape as their child.

```
         ┌────┐
         │ROOT│
         └─┬──┘
           │
  ┌────────┼───────┐
  │        │       │
┌─┴─┐    ┌─┴─┐   ┌─┴─┐
│T 1│    │T 2│   │TT3│
└─┬─┘    └─┬─┘   └─┬─┘
  │        │       │
┌─┴─┐    ┌─┴─┐   ┌─┴─┐
│SS1│    │SS2│   │SS3│
└───┘    └───┘   └───┘
```


In [ ]:
class Scene:

    @staticmethod
    def transform(shape, matrix):
        ret = []
        for point in shape:
            if len(point) == 2:
                point = list(point)
                point.append(1)
            elif len(point) != 3:
                raise ValueError("Points need to have a length of 2 or 3.")
            p = matrix @ point
            x = p.tolist()[0][0]
            y = p.tolist()[0][1]
            ret.append((x,y))
        return ret

    
    def __init__(self):
        self.root = None
        self.canvas = Image.new("L",(400,400))
        
    def draw_shape(self, shape, color=255):
        for p1, p2 in pairwise_wrap(shape):
            naive_line(self.canvas, map(int, p1), map(int, p2), color)
        
    def render(self):
        # a very simple, shallow scenegraph!
        transform_stack = []
        transform_stack.append(self.root.transform)
        for transform_node in self.root.children:
            transform_stack.append(transform_node.transform)         
            tm = transform_stack[0] @ transform_stack[1] # 0 = tm von root, 1 = tm des aktuellen Nodes
            for shape_node in transform_node.children:
                if type(shape_node) == Shape:
                    self.draw_shape(transform(shape_node.path,tm))
            transform_stack.pop()
        return self.canvas

    
class Node:

    def __init__(self, parent=None):
        self.parent = parent
        self.children = []
        self.transform = M([[1, 0, 0],
                            [0, 1, 0],
                            [0, 0, 1]])
        
    def add_child(self, child):
        self.children.append(child)
        
    def translate(self, x, y):
        tm = translate(x,y)
        self.transform = tm @ self.transform

        
class Shape:
    
    def __init__(self, path):
        self.path = path

In [ ]:
scene = Scene()
scene.root = Node()
rect = ((0,0), (0,100), (100,100), (100,0))

# Shape 1
shape1_t = Node()
scene.root.add_child(shape1_t)
shape1_t.add_child(Shape(rect))
shape1_t.translate(-100,-100)

# Shape 2
shape2_t = Node()
scene.root.add_child(shape2_t)
shape2_t.add_child(Shape(rect))
shape2_t.translate(0, 0)

scene.root.translate(200, 200)

scene.render()

## What's missing here?

(i.e., what could you add to this code)

- recursive parsing of the tree
- application of all transform matrices on the stack, not just the first two
- rotation / scaling
- a camera
- animation
- groups with inherited properties (such as color)